In [2]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
import re
from typing import Dict, List, Tuple, Optional

In [3]:
class NepaliEntityRecognition:
    def __init__(self):
        # Define entity types for e-commerce
        self.entity_types = {
            'PRODUCT': 0,
            'BRAND': 1,
            'PRICE': 2,
            'SIZE': 3,
            'COLOR': 4,
            'LOCATION': 5,
            'DATE': 6,
            'O': 7  # Other/Non-entity
        }

        # Training data: (sentence, [(word, entity_type)])
        self.training_data = [
            ("malai nike ko black shoes chaiyeko thiyo", [
                ("malai", "O"),
                ("nike", "BRAND"),
                ("ko", "O"),
                ("black", "COLOR"),
                ("shoes", "PRODUCT"),
                ("chaiyeko", "O"),
                ("thiyo", "O")
            ]),
            ("xs size ko red tshirt kati parcha", [
                ("xs", "SIZE"),
                ("size", "SIZE"),
                ("ko", "O"),
                ("red", "COLOR"),
                ("tshirt", "PRODUCT"),
                ("kati", "O"),
                ("parcha", "O")
            ])
        ]

        # Product catalog for entity validation
        self.catalog = {
            'products': ['shoes', 'tshirt', 'pants', 'jacket', 'bag'],
            'brands': ['nike', 'adidas', 'puma', 'reebok'],
            'colors': ['red', 'black', 'blue', 'white'],
            'sizes': ['xs', 's', 'm', 'l', 'xl']
        }

    def build_model(self, vocab_size: int, max_length: int) -> Sequential:
        """Build BiLSTM model for NER"""
        model = Sequential([
            Embedding(vocab_size, 64, input_length=max_length),
            Bidirectional(LSTM(32, return_sequences=True)),
            TimeDistributed(Dense(len(self.entity_types), activation='softmax'))
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def extract_entities(self, text: str) -> List[Tuple[str, str]]:
        """Extract entities from text using rules and catalog matching"""
        words = text.lower().split()
        entities = []

        for word in words:
            if word in self.catalog['products']:
                entities.append((word, 'PRODUCT'))
            elif word in self.catalog['brands']:
                entities.append((word, 'BRAND'))
            elif word in self.catalog['colors']:
                entities.append((word, 'COLOR'))
            elif word in self.catalog['sizes']:
                entities.append((word, 'SIZE'))
            elif re.match(r'rs\.?\s*\d+', word) or word.isdigit():
                entities.append((word, 'PRICE'))
            elif re.match(r'\d{1,2}/\d{1,2}/\d{2,4}', word):
                entities.append((word, 'DATE'))
            else:
                entities.append((word, 'O'))

        return entities

In [5]:
class ContextManager:
    def __init__(self):
        self.context = {}
        self.conversation_history = []
        self.current_intent = None

    def update_context(self, user_id: str, entities: List[Tuple[str, str]], intent: str):
        """Update context with new entities and intent"""
        if user_id not in self.context:
            self.context[user_id] = {
                'entities': {},
                'intent_history': [],
                'last_interaction': None
            }

        # Update entities in context
        for entity, entity_type in entities:
            if entity_type != 'O':
                self.context[user_id]['entities'][entity_type] = entity

        # Update intent history
        self.context[user_id]['intent_history'].append(intent)
        if len(self.context[user_id]['intent_history']) > 5:
            self.context[user_id]['intent_history'].pop(0)

    def get_context(self, user_id: str) -> Dict:
        """Get current context for user"""
        return self.context.get(user_id, {})

    def clear_context(self, user_id: str):
        """Clear context for user"""
        if user_id in self.context:
            del self.context[user_id]

In [6]:
class EnhancedNepaliChatbot:
    def __init__(self):
        self.ner = NepaliEntityRecognition()
        self.context_manager = ContextManager()

        # Enhanced response templates with entity placeholders
        self.response_templates = {
            'product_inquiry': [
                "{COLOR} {PRODUCT} ko price Rs. {PRICE} cha",
                "{BRAND} ko {PRODUCT} ahile stock ma cha",
                "{SIZE} size ko {PRODUCT} available cha"
            ],
            'delivery_inquiry': [
                "{LOCATION} ma delivery 3-5 din lagcha",
                "tapai ko order {DATE} ma deliver huncha"
            ]
        }

    def process_query(self, user_id: str, query: str) -> str:
        """Process user query with entity recognition and context"""
        # Extract entities
        entities = self.ner.extract_entities(query)

        # Determine intent (simplified for example)
        intent = self._determine_intent(query)

        # Update context
        self.context_manager.update_context(user_id, entities, intent)

        # Generate response using context
        return self._generate_response(user_id, entities, intent)

    def _determine_intent(self, query: str) -> str:
        """Simplified intent determination"""
        if any(word in query.lower() for word in ['price', 'paisa', 'kati']):
            return 'product_inquiry'
        elif any(word in query.lower() for word in ['delivery', 'shipping']):
            return 'delivery_inquiry'
        return 'general_inquiry'

    def _generate_response(self, user_id: str, entities: List[Tuple[str, str]], intent: str) -> str:
        """Generate response using context and entities"""
        context = self.context_manager.get_context(user_id)

        # Create entity dictionary from current entities
        entity_dict = {entity_type: entity for entity, entity_type in entities if entity_type != 'O'}

        # Add missing entities from context
        for entity_type, entity in context.get('entities', {}).items():
            if entity_type not in entity_dict:
                entity_dict[entity_type] = entity

        # Select response template
        if intent in self.response_templates:
            template = np.random.choice(self.response_templates[intent])

            # Fill template with entities
            try:
                response = template.format(**entity_dict)
            except KeyError:
                response = "Maaf garnuhos, thap jaankaari chaincha. Kripaya specific details dinuhos."
        else:
            response = "Maaf garnuhos, ma tapai ko prashna bujhina."

        return response

In [7]:
# Example usage
def main():
    chatbot = EnhancedNepaliChatbot()

    # Simulate conversation
    conversations = [
        ("nike ko black shoes kati parcha?", "user123"),
        ("tyo shoes xs size ma cha?", "user123"),
        ("kathmandu ma delivery kati din lagcha?", "user123")
    ]

    for query, user_id in conversations:
        print(f"\nUser: {query}")
        response = chatbot.process_query(user_id, query)
        print(f"Bot: {response}")

        # Print current context
        print("\nCurrent Context:")
        print(chatbot.context_manager.get_context(user_id))

In [8]:
# testing

main()


User: nike ko black shoes kati parcha?
Bot: Maaf garnuhos, thap jaankaari chaincha. Kripaya specific details dinuhos.

Current Context:
{'entities': {'BRAND': 'nike', 'COLOR': 'black', 'PRODUCT': 'shoes'}, 'intent_history': ['product_inquiry'], 'last_interaction': None}

User: tyo shoes xs size ma cha?
Bot: Maaf garnuhos, ma tapai ko prashna bujhina.

Current Context:
{'entities': {'BRAND': 'nike', 'COLOR': 'black', 'PRODUCT': 'shoes', 'SIZE': 'xs'}, 'intent_history': ['product_inquiry', 'general_inquiry'], 'last_interaction': None}

User: kathmandu ma delivery kati din lagcha?
Bot: Maaf garnuhos, thap jaankaari chaincha. Kripaya specific details dinuhos.

Current Context:
{'entities': {'BRAND': 'nike', 'COLOR': 'black', 'PRODUCT': 'shoes', 'SIZE': 'xs'}, 'intent_history': ['product_inquiry', 'general_inquiry', 'product_inquiry'], 'last_interaction': None}


In [9]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
import re
from typing import Dict, List, Tuple, Optional
from datetime import datetime, timedelta

class NepaliEntityRecognition:
    def __init__(self):
        self.entity_types = {
            'PRODUCT': 0,
            'BRAND': 1,
            'PRICE': 2,
            'SIZE': 3,
            'COLOR': 4,
            'LOCATION': 5,
            'DATE': 6,
            'O': 7
        }

        # Extended product catalog with prices
        self.catalog = {
            'products': {
                'shoes': {'price_range': (2000, 15000)},
                'tshirt': {'price_range': (500, 3000)},
                'pants': {'price_range': (1000, 5000)},
                'jacket': {'price_range': (2500, 8000)},
                'bag': {'price_range': (1000, 6000)}
            },
            'brands': {
                'nike': {'premium': True},
                'adidas': {'premium': True},
                'puma': {'premium': False},
                'reebok': {'premium': False}
            },
            'colors': ['red', 'black', 'blue', 'white'],
            'sizes': ['xs', 's', 'm', 'l', 'xl'],
            'locations': {
                'kathmandu': {'delivery_days': 2},
                'lalitpur': {'delivery_days': 2},
                'bhaktapur': {'delivery_days': 2},
                'pokhara': {'delivery_days': 4},
                'biratnagar': {'delivery_days': 5}
            }
        }

    def extract_entities(self, text: str) -> List[Tuple[str, str]]:
        words = text.lower().split()
        entities = []

        for word in words:
            if word in self.catalog['products']:
                entities.append((word, 'PRODUCT'))
            elif word in self.catalog['brands']:
                entities.append((word, 'BRAND'))
            elif word in self.catalog['colors']:
                entities.append((word, 'COLOR'))
            elif word in self.catalog['sizes']:
                entities.append((word, 'SIZE'))
            elif word in self.catalog['locations']:
                entities.append((word, 'LOCATION'))
            elif re.match(r'rs\.?\s*\d+', word) or word.isdigit():
                entities.append((word, 'PRICE'))
            elif re.match(r'\d{1,2}/\d{1,2}/\d{2,4}', word):
                entities.append((word, 'DATE'))
            else:
                entities.append((word, 'O'))

        return entities

    def get_product_price(self, product: str, brand: str) -> Tuple[int, int]:
        """Get price range for a product based on brand"""
        base_range = self.catalog['products'].get(product, {'price_range': (0, 0)})['price_range']
        if brand in self.catalog['brands'] and self.catalog['brands'][brand]['premium']:
            return (int(base_range[0] * 1.5), int(base_range[1] * 1.5))
        return base_range

class ContextManager:
    def __init__(self):
        self.context = {}
        self.conversation_history = []
        self.current_intent = None

    def update_context(self, user_id: str, entities: List[Tuple[str, str]], intent: str):
        if user_id not in self.context:
            self.context[user_id] = {
                'entities': {},
                'intent_history': [],
                'last_interaction': datetime.now()
            }

        # Update entities in context
        for entity, entity_type in entities:
            if entity_type != 'O':
                self.context[user_id]['entities'][entity_type] = entity

        # Update intent history
        self.context[user_id]['intent_history'].append(intent)
        if len(self.context[user_id]['intent_history']) > 5:
            self.context[user_id]['intent_history'].pop(0)

        self.context[user_id]['last_interaction'] = datetime.now()

class EnhancedNepaliChatbot:
    def __init__(self):
        self.ner = NepaliEntityRecognition()
        self.context_manager = ContextManager()

        # Enhanced response templates with more natural Romanized Nepali
        self.response_templates = {
            'product_inquiry': [
                "{BRAND} ko {COLOR} {PRODUCT} ko price Rs. {MIN_PRICE} dekhi Rs. {MAX_PRICE} sama cha.",
                "{COLOR} {PRODUCT} ko price range Rs. {MIN_PRICE} - {MAX_PRICE} ho.",
                "{BRAND} ko {PRODUCT} ko price range Rs. {MIN_PRICE} - {MAX_PRICE} cha."
            ],
            'stock_inquiry': [
                "{BRAND} ko {COLOR} {PRODUCT} {SIZE} size ma available cha.",
                "{SIZE} size ko {COLOR} {PRODUCT} stock ma cha.",
                "Hajur, {COLOR} {PRODUCT} {SIZE} size ma paunus."
            ],
            'delivery_inquiry': [
                "{LOCATION} ma delivery {DAYS} din ma huncha.",
                "{LOCATION} ma parcel pugnalai {DAYS} din lagcha.",
                "Hajur ko {LOCATION} ko location ma delivery {DAYS} din bhitra huncha."
            ]
        }

    def _determine_intent(self, query: str) -> str:
        """Enhanced intent determination"""
        query = query.lower()
        if any(word in query for word in ['kati', 'price', 'paisa', 'rate']):
            return 'product_inquiry'
        elif any(word in query for word in ['size', 'available', 'cha', 'stock']):
            return 'stock_inquiry'
        elif any(word in query for word in ['delivery', 'shipping', 'pugihalcha', 'aipugcha']):
            return 'delivery_inquiry'
        return 'general_inquiry'

    def _generate_response(self, user_id: str, entities: List[Tuple[str, str]], intent: str) -> str:
        context = self.context_manager.get_context(user_id)
        entity_dict = {entity_type: entity for entity, entity_type in entities if entity_type != 'O'}

        # Add missing entities from context
        for entity_type, entity in context.get('entities', {}).items():
            if entity_type not in entity_dict:
                entity_dict[entity_type] = entity

        # Generate appropriate response based on intent and available entities
        if intent == 'product_inquiry':
            if 'PRODUCT' in entity_dict:
                product = entity_dict.get('PRODUCT')
                brand = entity_dict.get('BRAND', 'standard')
                min_price, max_price = self.ner.get_product_price(product, brand)
                entity_dict['MIN_PRICE'] = min_price
                entity_dict['MAX_PRICE'] = max_price
                template = np.random.choice(self.response_templates['product_inquiry'])
                return template.format(**entity_dict)

        elif intent == 'stock_inquiry':
            if all(key in entity_dict for key in ['PRODUCT', 'SIZE']):
                template = np.random.choice(self.response_templates['stock_inquiry'])
                return template.format(**entity_dict)

        elif intent == 'delivery_inquiry':
            if 'LOCATION' in entity_dict:
                location = entity_dict['LOCATION']
                days = self.ner.catalog['locations'].get(location, {'delivery_days': 5})['delivery_days']
                entity_dict['DAYS'] = days
                template = np.random.choice(self.response_templates['delivery_inquiry'])
                return template.format(**entity_dict)

        # If we can't generate a specific response, ask for more details
        missing_info = self._get_missing_info(intent, entity_dict)
        return f"Kripaya {missing_info} specify garnuhos."

    def _get_missing_info(self, intent: str, entities: Dict) -> str:
        """Determine what information is missing for a complete response"""
        if intent == 'product_inquiry':
            if 'PRODUCT' not in entities:
                return "kun product"
            if 'BRAND' not in entities:
                return "kun brand"
        elif intent == 'stock_inquiry':
            if 'PRODUCT' not in entities:
                return "kun product"
            if 'SIZE' not in entities:
                return "kun size"
        elif intent == 'delivery_inquiry':
            if 'LOCATION' not in entities:
                return "delivery location"
        return "thap details"

    def process_query(self, user_id: str, query: str) -> str:
        entities = self.ner.extract_entities(query)
        intent = self._determine_intent(query)
        self.context_manager.update_context(user_id, entities, intent)
        return self._generate_response(user_id, entities, intent)

# Example usage with the same queries
def main():
    chatbot = EnhancedNepaliChatbot()

    conversations = [
        ("nike ko black shoes kati parcha?", "user123"),
        ("tyo shoes xs size ma cha?", "user123"),
        ("kathmandu ma delivery kati din lagcha?", "user123")
    ]

    for query, user_id in conversations:
        print(f"\nUser: {query}")
        response = chatbot.process_query(user_id, query)
        print(f"Bot: {response}")
        print("\nCurrent Context:")
        print(chatbot.context_manager.get_context(user_id))

if __name__ == "__main__":
    main()


User: nike ko black shoes kati parcha?


AttributeError: 'ContextManager' object has no attribute 'get_context'

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle
import json

class NepaliChatbotTrainer:
    def __init__(self, max_words=5000, max_len=50):
        self.max_words = max_words
        self.max_len = max_len
        self.tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")

        # Training data for intent classification
        self.intent_data = {
            "product_inquiry": [
                "yo product ko price kati ho",
                "kati parcha",
                "rate kati cha",
                "price kati ho",
                "yesko dam kati ho",
                "nike ko shoes kati parcha",
                "tshirt ko price kati ho",
                "jacket ko rate bhannus na",
                "pants ko price thaha cha"
            ],
            "stock_inquiry": [
                "stock ma cha",
                "available cha",
                "xs size cha",
                "large size paucha",
                "red color cha",
                "black shoes cha",
                "blue tshirt available cha",
                "nike ko shoes stock ma cha"
            ],
            "delivery_inquiry": [
                "delivery kati din ma huncha",
                "shipping time kati ho",
                "kahile samma aipugcha",
                "kathmandu ma kati din ma aaucha",
                "delivery charge kati ho",
                "shipping cost kati parcha",
                "delivery free cha"
            ]
        }

        # Training data for NER
        self.ner_data = [
            ("nike ko black shoes kati parcha", [
                ("nike", "BRAND"),
                ("ko", "O"),
                ("black", "COLOR"),
                ("shoes", "PRODUCT"),
                ("kati", "O"),
                ("parcha", "O")
            ]),
            ("red tshirt xs size ma cha", [
                ("red", "COLOR"),
                ("tshirt", "PRODUCT"),
                ("xs", "SIZE"),
                ("size", "O"),
                ("ma", "O"),
                ("cha", "O")
            ]),
            ("kathmandu ma delivery kati din lagcha", [
                ("kathmandu", "LOCATION"),
                ("ma", "O"),
                ("delivery", "O"),
                ("kati", "O"),
                ("din", "O"),
                ("lagcha", "O")
            ])
        ]

    def prepare_intent_data(self):
        """Prepare data for intent classification training"""
        texts = []
        labels = []
        label_to_id = {}

        for idx, (intent, sentences) in enumerate(self.intent_data.items()):
            label_to_id[intent] = idx
            for sentence in sentences:
                texts.append(sentence)
                labels.append(idx)

        # Tokenize texts
        self.tokenizer.fit_on_texts(texts)
        sequences = self.tokenizer.texts_to_sequences(texts)
        X = pad_sequences(sequences, maxlen=self.max_len)
        y = to_categorical(labels)

        return X, y, label_to_id

    def prepare_ner_data(self):
        """Prepare data for NER training"""
        texts = []
        labels = []

        # Create word and tag vocabularies
        words = set()
        tags = set()

        for sentence, annotations in self.ner_data:
            words.update([word for word, _ in annotations])
            tags.update([tag for _, tag in annotations])

        word_to_id = {word: idx for idx, word in enumerate(words, 1)}
        tag_to_id = {tag: idx for idx, tag in enumerate(tags)}

        # Convert to sequences
        for sentence, annotations in self.ner_data:
            text_seq = [word_to_id.get(word, 0) for word, _ in annotations]
            label_seq = [tag_to_id[tag] for _, tag in annotations]

            texts.append(text_seq)
            labels.append(label_seq)

        # Pad sequences
        X = pad_sequences(texts, maxlen=self.max_len, padding='post')
        y = pad_sequences(labels, maxlen=self.max_len, padding='post')
        y = to_categorical(y, num_classes=len(tag_to_id))

        return X, y, word_to_id, tag_to_id

    def build_intent_model(self, num_classes):
        """Build intent classification model"""
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(self.max_words, 128, input_length=self.max_len),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

        model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
        return model

    def build_ner_model(self, vocab_size, num_tags):
        """Build NER model"""
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size + 1, 128, input_length=self.max_len),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64, activation='relu')),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_tags, activation='softmax'))
        ])

        model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
        return model

    def train_models(self):
        """Train both intent and NER models"""
        # Train intent classification model
        X_intent, y_intent, intent_label_map = self.prepare_intent_data()
        X_train_intent, X_test_intent, y_train_intent, y_test_intent = train_test_split(
            X_intent, y_intent, test_size=0.2, random_state=42
        )

        intent_model = self.build_intent_model(len(intent_label_map))
        intent_history = intent_model.fit(
            X_train_intent, y_train_intent,
            epochs=10,
            batch_size=32,
            validation_data=(X_test_intent, y_test_intent),
            verbose=1
        )

        # Train NER model
        X_ner, y_ner, word_map, tag_map = self.prepare_ner_data()
        X_train_ner, X_test_ner, y_train_ner, y_test_ner = train_test_split(
            X_ner, y_ner, test_size=0.2, random_state=42
        )

        ner_model = self.build_ner_model(len(word_map), len(tag_map))
        ner_history = ner_model.fit(
            X_train_ner, y_train_ner,
            epochs=10,
            batch_size=32,
            validation_data=(X_test_ner, y_test_ner),
            verbose=1
        )

        # Save models and mappings
        self.save_models(intent_model, ner_model,
                        intent_label_map, word_map, tag_map)

        return intent_history, ner_history

    def save_models(self, intent_model, ner_model,
                   intent_label_map, word_map, tag_map):
        """Save trained models and mappings"""
        # Save models
        intent_model.save('intent_model.h5')
        ner_model.save('ner_model.h5')

        # Save tokenizer
        with open('tokenizer.pickle', 'wb') as handle:
            pickle.dump(self.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

        # Save mappings
        mappings = {
            'intent_label_map': intent_label_map,
            'word_map': word_map,
            'tag_map': tag_map
        }
        with open('mappings.json', 'w', encoding='utf-8') as f:
            json.dump(mappings, f, ensure_ascii=False, indent=2)

def main():
    # Initialize and train the models
    trainer = NepaliChatbotTrainer()
    intent_history, ner_history = trainer.train_models()

    # Print training results
    print("\nIntent Classification Training Results:")
    print(f"Final accuracy: {intent_history.history['accuracy'][-1]:.4f}")
    print(f"Final validation accuracy: {intent_history.history['val_accuracy'][-1]:.4f}")

    print("\nNER Training Results:")
    print(f"Final accuracy: {ner_history.history['accuracy'][-1]:.4f}")
    print(f"Final validation accuracy: {ner_history.history['val_accuracy'][-1]:.4f}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.3684 - loss: 1.0988 - val_accuracy: 0.4000 - val_loss: 1.0889
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step - accuracy: 0.3684 - loss: 1.0889 - val_accuracy: 0.4000 - val_loss: 1.0846
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.3684 - loss: 1.0928 - val_accuracy: 0.4000 - val_loss: 1.0795
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.4211 - loss: 1.0737 - val_accuracy: 0.4000 - val_loss: 1.0723
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.4211 - loss: 1.0696 - val_accuracy: 0.4000 - val_loss: 1.0671
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.3684 - loss: 1.0863 - val_accuracy: 0.4000 - val_loss: 1.0640
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.3684 - loss: 1.0682 - val_accuracy: 0.4000 - val_loss: 1.0611
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.4737 - loss: 1.0704 - val_accuracy: 0.4000 - val_loss: 1.


Intent Classification Training Results:
Final accuracy: 0.3684
Final validation accuracy: 0.4000

NER Training Results:
Final accuracy: 0.9600
Final validation accuracy: 0.9400


In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
import pickle

class ChatbotTester:
    def __init__(self):
        # Load trained models
        try:
            self.intent_model = tf.keras.models.load_model('intent_model.h5')
            self.ner_model = tf.keras.models.load_model('ner_model.h5')

            # Load tokenizer
            with open('tokenizer.pickle', 'rb') as handle:
                self.tokenizer = pickle.load(handle)

            # Load mappings
            with open('mappings.json', 'r', encoding='utf-8') as f:
                self.mappings = json.load(f)

            self.id_to_intent = {v: k for k, v in self.mappings['intent_label_map'].items()}
            self.id_to_tag = {v: k for k, v in self.mappings['tag_map'].items()}

            print("Models and mappings loaded successfully!")

        except Exception as e:
            print(f"Error loading models: {e}")
            print("\nFirst run the training script to generate the models!")
            return

        # Test cases
        self.test_cases = [
            # Product inquiries
            "nike ko black shoes kati parcha",
            "red tshirt ko price kati ho",
            "adidas ko jacket ko rate bhannus na",
            "xs size ko pant kati parcha",

            # Stock inquiries
            "black nike shoes xs size ma cha",
            "l size ko red tshirt available cha",
            "white adidas shoes stock ma cha",
            "blue color ko jacket paucha",

            # Delivery inquiries
            "kathmandu ma delivery kati din ma huncha",
            "pokhara ma shipping kati din lagcha",
            "delhi ma pugihalcha",
            "shipping charge kati ho",

            # Mixed queries
            "nike ko black shoes xs size ma cha ani price kati ho",
            "kathmandu ma delivery free cha? ani kati din ma aaucha?",

            # Edge cases
            "k cha",
            "namaste",
            "dhanyabaad",
            "ma kinchu"
        ]

    def predict_intent(self, text):
        """Predict intent for given text"""
        # Tokenize and pad the text
        sequence = self.tokenizer.texts_to_sequences([text])
        padded = pad_sequences(sequence, maxlen=50)

        # Predict intent
        prediction = self.intent_model.predict(padded)[0]
        intent_id = np.argmax(prediction)
        confidence = prediction[intent_id]

        return self.id_to_intent[intent_id], confidence

    def predict_entities(self, text):
        """Predict entities in given text"""
        # Convert text to sequence using word_map
        words = text.split()
        word_ids = [self.mappings['word_map'].get(word, 0) for word in words]
        padded = pad_sequences([word_ids], maxlen=50, padding='post')

        # Predict entities
        predictions = self.ner_model.predict(padded)[0]

        entities = []
        for i, word in enumerate(words):
            if i < len(predictions):
                tag_id = np.argmax(predictions[i])
                confidence = predictions[i][tag_id]
                if tag_id in self.id_to_tag:
                    tag = self.id_to_tag[tag_id]
                    if tag != 'O':  # Only include named entities
                        entities.append({
                            'word': word,
                            'entity': tag,
                            'confidence': float(confidence)
                        })

        return entities

    def run_tests(self):
        """Run tests on all test cases"""
        print("\nRunning tests...\n")
        print("=" * 80)

        for i, test_case in enumerate(self.test_cases, 1):
            print(f"\nTest Case {i}: '{test_case}'")
            print("-" * 40)

            # Predict intent
            intent, confidence = self.predict_intent(test_case)
            print(f"Intent: {intent} (confidence: {confidence:.4f})")

            # Predict entities
            entities = self.predict_entities(test_case)
            if entities:
                print("\nEntities found:")
                for entity in entities:
                    print(f"- {entity['word']}: {entity['entity']} "
                          f"(confidence: {entity['confidence']:.4f})")
            else:
                print("\nNo entities found")

            print("-" * 40)

        print("\nTest completion summary:")
        print("=" * 80)

    def interactive_test(self):
        """Interactive testing mode"""
        print("\nEntering interactive testing mode (type 'exit' to quit)")
        print("=" * 80)

        while True:
            query = input("\nEnter your query: ")
            if query.lower() == 'exit':
                break

            print("\nAnalyzing query...")
            print("-" * 40)

            # Predict intent
            intent, confidence = self.predict_intent(query)
            print(f"Intent: {intent} (confidence: {confidence:.4f})")

            # Predict entities
            entities = self.predict_entities(query)
            if entities:
                print("\nEntities found:")
                for entity in entities:
                    print(f"- {entity['word']}: {entity['entity']} "
                          f"(confidence: {entity['confidence']:.4f})")
            else:
                print("\nNo entities found")

            print("-" * 40)

def main():
    tester = ChatbotTester()

    # Run automated tests
    tester.run_tests()

    # Start interactive testing
    print("\nWould you like to try interactive testing? (yes/no)")
    response = input()
    if response.lower() in ['yes', 'y']:
        tester.interactive_test()

if __name__ == "__main__":
    main()

Models and mappings loaded successfully!

Running tests...


Test Case 1: 'nike ko black shoes kati parcha'
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
Intent: product_inquiry (confidence: 0.3989)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

No entities found
----------------------------------------

Test Case 2: 'red tshirt ko price kati ho'
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Intent: product_inquiry (confidence: 0.4112)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

No entities found
----------------------------------------

Test Case 3: 'adidas ko jacket ko rate bhannus na'
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Intent: product_inquiry (confidence: 0.4037)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

No entities found
----------------------------------------

Test Case 4: 'xs size ko pant kati parcha'
----------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Intent: 